In [ ]:
import os
import chainlit as cl
import google.generativeai as genai
from gemini_cell_agent.utils import exec_code, append_image_metadata
from gemini_cell_agent.sys_prompt import pkg_sys_prompt

import datetime

context_file = genai.upload_file(path='../data/cellpose.txt')
sys_prompt = pkg_sys_prompt('cellpose')

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model_name = "gemini-1.5-flash"
config = genai.GenerationConfig(temperature=0)

# Create a cache with a 5 minute TTL
cache = genai.caching.CachedContent.create(
    model=f'models/{model_name}-001',
    display_name='cellpose github repo content', # used to identify the cache
    system_instruction=sys_prompt,
    contents=[context_file],
    ttl=datetime.timedelta(minutes=10),
)

model = genai.GenerativeModel.from_cached_content(cached_content=cache, generation_config=config)